In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "arcos_all_washpost.zip", compression="zip", header=0, sep="\t", nrows=10
)
df

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
0,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-02-08,"Par Pharmaceutical, Inc.",McKesson Corporation,0.896500,100.0,1344.7500
1,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,7.5,2011-03-07,SpecGx LLC,McKesson Corporation,0.454050,100.0,454.0500
2,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-03-10,"Par Pharmaceutical, Inc.",McKesson Corporation,3.586000,400.0,5379.0000
3,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-05,SpecGx LLC,McKesson Corporation,1.513500,500.0,1513.5000
4,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-06,SpecGx LLC,McKesson Corporation,0.302700,100.0,302.7000
5,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-04-05,"Par Pharmaceutical, Inc.",McKesson Corporation,1.793000,200.0,2689.5000
6,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-04-05,"Par Pharmaceutical, Inc.",McKesson Corporation,0.896500,100.0,1344.7500
7,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,7.5,2011-09-27,SpecGx LLC,McKesson Corporation,0.908100,200.0,908.1000
8,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,10.0,2012-04-11,SpecGx LLC,McKesson Corporation,3.027000,500.0,3027.0000
9,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,7.5,2012-05-08,Actavis,McKesson Corporation,0.672375,100.0,1008.5625


In [3]:
required_columns = [
    "BUYER_COUNTY",
    "BUYER_STATE",
    "TRANSACTION_DATE",
    "MME",
]

In [6]:
# only print df state
df["BUYER_STATE"].value_counts()

BUYER_STATE
IN    10
Name: count, dtype: int64

In [26]:
chunksize = 10**6
chunks = []

for chunk in pd.read_csv(
    "arcos_all_washpost.zip", compression="zip", sep="\t", chunksize=chunksize
):
    chunk = chunk[required_columns]
    chunk = chunk[
        chunk["BUYER_STATE"].isin(["TX", "AL", "SC", "TN"])
    ]  # Filter for Texas and control states
    chunk["Year"] = pd.to_datetime(chunk["TRANSACTION_DATE"], format="%Y-%m-%d").dt.year
    chunk = chunk[chunk["Year"].isin([2006, 2007, 2008])]  # Filter for 2006, 2007, 2008
    chunk["Month"] = pd.to_datetime(
        chunk["TRANSACTION_DATE"], format="%Y-%m-%d"
    ).dt.month
    chunk_grouped = (
        chunk.groupby(["BUYER_COUNTY", "BUYER_STATE", "Month", "Year"])[["MME"]]
        .sum()
        .reset_index()
    )
    chunks.append(chunk_grouped)

df = pd.concat(chunks)

/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/4048744379.py:4: DtypeWarning: Columns (3,5,22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(
/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/4048744379.py:4: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(
/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/4048744379.py:4: DtypeWarning: Columns (3,5,22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(
/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/4048744379.py:4: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(
/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/4048744379.py:4: DtypeWarning: Columns (22) have mixed types. Specify dtype option 

In [27]:
df

,BUYER_COUNTY,BUYER_STATE,Month,Year,MME
0,ANDERSON,TX,1,2006,12713.4000
1,ANDERSON,TX,1,2007,9989.1000
2,ANDERSON,TX,1,2008,9535.0500
3,ANDERSON,TX,2,2006,5599.9500
4,ANDERSON,TX,2,2007,2572.9500
...,...,...,...,...,...
1643,YORK,SC,11,2007,291364.1930
1644,YORK,SC,11,2008,171297.2925
1645,YORK,SC,12,2006,269494.7880
1646,YORK,SC,12,2007,262956.4965


In [28]:
# create a copy of df

df_copy = df.copy()

In [30]:
df = (
    df.groupby(["BUYER_COUNTY", "BUYER_STATE", "Month", "Year"])[["MME"]]
    .sum()
    .reset_index()
)

In [31]:
df

,BUYER_COUNTY,BUYER_STATE,Month,Year,MME
0,ABBEVILLE,SC,1,2006,243682.3050
1,ABBEVILLE,SC,1,2007,346518.0735
2,ABBEVILLE,SC,1,2008,139261.9770
3,ABBEVILLE,SC,2,2006,192223.9800
4,ABBEVILLE,SC,2,2007,335250.1305
...,...,...,...,...,...
15483,ZAVALA,TX,11,2007,61599.4500
15484,ZAVALA,TX,11,2008,60388.6500
15485,ZAVALA,TX,12,2006,48583.3500
15486,ZAVALA,TX,12,2007,77642.5500


In [47]:
# load population data

population = pd.read_csv("final_population_data.csv")
copy = population.copy()

In [48]:
population

,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,G0100010,1,1,Autauga County,Alabama,46348,47458,47917,49039,49834,51413,52992,54571,54580,54590,55384,55982,56334
1,G0100030,1,3,Baldwin County,Alabama,151574,156688,162261,168154,172815,175965,179115,182265,182745,183226,195540,200111,203709
2,G0100050,1,5,Barbour County,Alabama,28805,28545,28302,29420,29736,28976,28216,27457,27463,27469,27092,26811,26649
3,G0100070,1,7,Bibb County,Alabama,21224,21290,21397,21307,21485,21961,22438,22915,22842,22769,22971,23104,23151
4,G0100090,1,9,Blount County,Alabama,54195,54973,54983,55708,56866,57018,57170,57322,57394,57466,57578,57680,57769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,G5600370,56,37,Sweetwater County,Wyoming,37155,37570,37331,38001,39320,40815,42310,43806,43848,43890,44658,45245,45604
3139,G5600390,56,39,Teton County,Wyoming,18702,19001,19327,19588,20073,20480,20887,21294,21310,21326,21537,21699,21800
3140,G5600410,56,41,Uinta County,Wyoming,19756,19786,19451,19688,20071,20420,20769,21118,21030,20942,21072,21147,21154
3141,G5600430,56,43,Washakie County,Wyoming,7923,7890,7773,7675,7805,8047,8290,8533,8479,8425,8519,8576,8586


In [49]:
# remove GISJOIN column, county_code and state_code

population = population.drop(columns=["GISJOIN", "STATE_CODE", "COUNTY_CODE"])

In [50]:
# Turn the different years columns in to rows

population = pd.melt(
    population, id_vars=["COUNTY", "STATE"], var_name="Year", value_name="Population"
)

In [56]:
# only keep the states we want

population = population[
    population["STATE"].isin(["Texas", "Alabama", "South Carolina", "Tennessee"])
]

In [58]:
population

,COUNTY,STATE,Year,Population
9429,Autauga County,Alabama,2006,49039
9430,Baldwin County,Alabama,2006,168154
9431,Barbour County,Alabama,2006,29420
9432,Bibb County,Alabama,2006,21307
9433,Blount County,Alabama,2006,55708
...,...,...,...,...
18487,Wood County,Texas,2008,42168
18488,Yoakum County,Texas,2008,7577
18489,Young County,Texas,2008,17999
18490,Zapata County,Texas,2008,13703


In [54]:
# only keep 2006, 2007, 2008 for population

population = population[population["Year"].isin(["2006", "2007", "2008"])]

In [61]:
# remove the word county from the COUNTY column

population["COUNTY"] = population["COUNTY"].str.replace(" County", "")

# make the string lower

population["COUNTY"] = population["COUNTY"].str.lower()

/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/3928866853.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population["COUNTY"] = population["COUNTY"].str.replace(" County", "")
/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/3928866853.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population["COUNTY"] = population["COUNTY"].str.lower()


In [62]:
population

,COUNTY,STATE,Year,Population
9429,autauga,Alabama,2006,49039
9430,baldwin,Alabama,2006,168154
9431,barbour,Alabama,2006,29420
9432,bibb,Alabama,2006,21307
9433,blount,Alabama,2006,55708
...,...,...,...,...
18487,wood,Texas,2008,42168
18488,yoakum,Texas,2008,7577
18489,young,Texas,2008,17999
18490,zapata,Texas,2008,13703


In [70]:
state_dict = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
state_dict = dict(map(reversed, state_dict.items()))

In [72]:
state_dict

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming',
 'DC': 'District of Columbia

In [67]:
# add state code column to population

population["STATE_CODE"] = population["STATE"].map(state_dict)

/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/3329161507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population["STATE_CODE"] = population["STATE"].map(state_dict)


In [75]:
df["State"] = df["BUYER_STATE"].map(state_dict)
# shipments['State Name'] = shipments['BUYER_STATE'].map(state_dict)

# lowercase for buyer county

df["BUYER_COUNTY"] = df["BUYER_COUNTY"].str.lower()

In [79]:
# turn population year into int

population["Year"] = population["Year"].astype(int)

/var/folders/mm/1ns7h3xd5h71r5vbw2p8jkm80000gn/T/ipykernel_62001/600223067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['Year'] = population['Year'].astype(int)


In [80]:
# now merge df and population

df_merged = pd.merge(
    df,
    population,
    how="left",
    left_on=["BUYER_COUNTY", "State", "Year"],
    right_on=["COUNTY", "STATE", "Year"],
)

In [82]:
# clean up the columns

df_merged = df_merged.drop(columns=["STATE_CODE", "STATE"])

In [84]:
# find MME per capita

df_merged["MME_rate"] = df_merged["MME"] / df_merged["Population"]

In [85]:
df_merged

,BUYER_COUNTY,BUYER_STATE,Month,Year,MME,State,COUNTY,Population,MME_rate
0,abbeville,SC,1,2006,243682.3050,South Carolina,abbeville,25497.0,9.557293
1,abbeville,SC,1,2007,346518.0735,South Carolina,abbeville,25318.0,13.686629
2,abbeville,SC,1,2008,139261.9770,South Carolina,abbeville,25351.0,5.493352
3,abbeville,SC,2,2006,192223.9800,South Carolina,abbeville,25497.0,7.539082
4,abbeville,SC,2,2007,335250.1305,South Carolina,abbeville,25318.0,13.241572
...,...,...,...,...,...,...,...,...,...
15483,zavala,TX,11,2007,61599.4500,Texas,zavala,11640.0,5.292049
15484,zavala,TX,11,2008,60388.6500,Texas,zavala,11652.0,5.182685
15485,zavala,TX,12,2006,48583.3500,Texas,zavala,11605.0,4.186415
15486,zavala,TX,12,2007,77642.5500,Texas,zavala,11640.0,6.670322


In [88]:
# save to csv

df_merged.to_csv("monthly.csv")

In [89]:
df_merged

,BUYER_COUNTY,BUYER_STATE,Month,Year,MME,State,COUNTY,Population,MME_rate
0,abbeville,SC,1,2006,243682.3050,South Carolina,abbeville,25497.0,9.557293
1,abbeville,SC,1,2007,346518.0735,South Carolina,abbeville,25318.0,13.686629
2,abbeville,SC,1,2008,139261.9770,South Carolina,abbeville,25351.0,5.493352
3,abbeville,SC,2,2006,192223.9800,South Carolina,abbeville,25497.0,7.539082
4,abbeville,SC,2,2007,335250.1305,South Carolina,abbeville,25318.0,13.241572
...,...,...,...,...,...,...,...,...,...
15483,zavala,TX,11,2007,61599.4500,Texas,zavala,11640.0,5.292049
15484,zavala,TX,11,2008,60388.6500,Texas,zavala,11652.0,5.182685
15485,zavala,TX,12,2006,48583.3500,Texas,zavala,11605.0,4.186415
15486,zavala,TX,12,2007,77642.5500,Texas,zavala,11640.0,6.670322
